### This is the minimal amount of code for you to start classifying sentiment to 1 (negative) or 2 (positive)
To find out more, please follow fastai's lesson 4:
https://github.com/fastai/fastai/blob/master/courses/dl1/lesson4-imdb.ipynb?short_path=41823fc

用最少代码运行语言情感分析，借用fastai已有的语义分析模型，通过IMDB数据库训练，迁移到跟比特币和区块链相关的语义情感分析上。
如果想看训练的代码，请跳转到fastai

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

# fastai libraries
from fastai.learner import partial, optim, accuracy_np, to_np, torch
from fastai.nlp import TextData
from fastai.lm_rnn import seq2seq_reg


import dill as pickle
import spacy

In [2]:
spacy_tok = spacy.load('en')

In [3]:
TEXT = data.Field(lower=True, tokenize="spacy")

Download pickle file here: https://drive.google.com/file/d/1e3Bq9VzDsS_NGxyiY1QyEGXSnccuHFhJ/view?usp=sharing

Put this in the project dir

In [4]:
PATH='./'
TEXT = pickle.load(open(f'{PATH}/TEXT.pkl','rb'))

In [5]:
TEXT.vocab.itos[:12]

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is', 'in', 'it']

The first time you run this, torchtext will download IMDB data to project folder

In [6]:
IMDB_LABEL = data.Field(sequential=False)
splits = torchtext.datasets.IMDB.splits(TEXT, IMDB_LABEL, './')

downloading aclImdb_v1.tar.gz


Download cycle file here: https://drive.google.com/open?id=1yqIXOYTXpS5fSyzQlyYxnN6djp27EzGa

Download encoder file here: https://drive.google.com/open?id=16l7RR8zVJp7tukFOAH778rdi4ntO9_7i

put these two files inside `models` dir

In [8]:
bs=56        # batch size
bptt=70      # batch per token
em_sz = 200  # size of each embedding vector
nh = 500     # number of hidden activations per layer
nl = 3       # number of layers
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))
md2 = TextData.from_splits(PATH, splits, bs)
m3 = md2.get_model(opt_fn, 1500, bptt, emb_sz=em_sz, n_hid=nh, n_layers=nl, 
           dropout=0.1, dropouti=0.4, wdrop=0.5, dropoute=0.05, dropouth=0.3)
m3.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
m3.load_encoder(f'adam3_20_enc')
m3.load_cycle('imdb2', 4)

Check accuracy, should be about 0.93
Skip this check if your laptop is too slow

In [9]:
accuracy_np(*m3.predict_with_targs())

0.9332

In [10]:
m=m3.model

test_str ="this is good for bitcoin"
tokenized_str = spacy_tok(test_str)
token_lst = [sent.string.strip() for sent in tokenized_str]

t = TEXT.numericalize([token_lst])
print(t)

# Set batch size to 1
m[0].bs=1

# Turn off dropout
m.eval()
# Reset hidden state
m.reset()
# Get predictions from model
res,*_ = m(t)
num_res = to_np(torch.topk(res[-1], 1)[1])[0]

print(res)
print(num_res)

Variable containing:
 13
  9
 62
 22
  0
[torch.cuda.LongTensor of size 5x1 (GPU 0)]

Variable containing:
-8.8046 -2.2182  2.5809
[torch.cuda.FloatTensor of size 1x3 (GPU 0)]

2


1 (negative) or 2 (positive)